In [ ]:
import os

folder_path = '/Users/suryanshpatel/Projects/Directed Readings/Technical/MintPain/Annotated_data_part1/Sub1 Daniel Simonsen/Annotated_data_Sub01_Trial01/Sub01_Trial01_Sweep01_Label0/RGB'  # replace with your folder path
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(f"Number of files: {len(files)}")


Number of files: 28


: 

# Script to display RGB T D 

In [2]:
import os
import cv2
import numpy as np
from glob import glob
from pathlib import Path
from skimage.transform import ProjectiveTransform

# Define Paths
data_dir = "/Users/suryanshpatel/Projects/Directed Readings/Technical/MintPain/Annotated_data_part2"
subjects = sorted([subject for subject in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, subject))])

# Initialize Face Detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Homography Matrix Placeholder (Needs to be precomputed and loaded)
homography_rgb_to_thermal = np.eye(3)  # Replace with actual matrix
homography_rgb_to_depth = np.eye(3)  # Replace with actual matrix

for subject in subjects:
    subject_path = os.path.join(data_dir, subject)
    trials = sorted([trial for trial in os.listdir(subject_path) if os.path.isdir(os.path.join(subject_path, trial))])
    
    for trial in trials:
        trial_path = os.path.join(subject_path, trial)
        sweeps = sorted([sweep for sweep in os.listdir(trial_path) if os.path.isdir(os.path.join(trial_path, sweep))])
        
        for sweep in sweeps:
            sweep_path = os.path.join(trial_path, sweep)
            
            # Load RGB, Depth, and Thermal images with updated folder names
            rgb_images = sorted(glob(os.path.join(sweep_path, "RGB", "*.jpg")))
            depth_images = sorted(glob(os.path.join(sweep_path, "D", "*.png")))  # Corrected for "D"
            thermal_images = sorted(glob(os.path.join(sweep_path, "T", "*.png")))  # Corrected for "T"
            
            # Check if depth and thermal images exist
            if len(depth_images) == 0 or len(thermal_images) == 0:
                print(f"No depth or thermal images found for sweep: {sweep_path}")
                continue
            
            for rgb_path in rgb_images:
                # Extract timestamps from filenames (assuming same format as MATLAB script)
                filename = Path(rgb_path).stem
                print(f"Processing: {filename}")
                frame_num = int(filename.split('-')[-1])  # Adjust parsing based on actual format
                
                # Ensure frame_num is within bounds for both depth and thermal images
                depth_idx = min(frame_num, len(depth_images) - 1)
                thermal_idx = min(frame_num, len(thermal_images) - 1)
                
                # Print lengths and indices for debugging
                print(f"Total depth images: {len(depth_images)}")
                print(f"Total thermal images: {len(thermal_images)}")
                print(f"Requested frame_num: {frame_num}")
                print(f"Using depth_idx: {depth_idx}, thermal_idx: {thermal_idx}")
                
                # Ensure indices are valid
                if depth_idx >= len(depth_images) or thermal_idx >= len(thermal_images):
                    print(f"Index out of range! Skipping {rgb_path}")
                    continue
                
                # Find corresponding Depth and Thermal images
                depth_path = depth_images[depth_idx]
                thermal_path = thermal_images[thermal_idx]
                
                # Check if images exist before proceeding
                if not os.path.exists(depth_path) or not os.path.exists(thermal_path):
                    print(f"Missing depth or thermal image for {rgb_path}")
                    continue
                
                # Read images
                rgb_img = cv2.imread(rgb_path)
                depth_img = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)
                thermal_img = cv2.imread(thermal_path, cv2.IMREAD_GRAYSCALE)
                
                # Face Detection in RGB
                gray_rgb = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray_rgb, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
                
                if len(faces) > 0:
                    x, y, w, h = faces[0]  # Assuming first face is the correct one
                    
                    # Project bounding box using homography
                    rgb_bbox = np.array([[x, y, 1], [x+w, y+h, 1]]).T
                    
                    thermal_bbox = np.dot(homography_rgb_to_thermal, rgb_bbox)
                    thermal_bbox /= thermal_bbox[2]  # Normalize
                    
                    depth_bbox = np.dot(homography_rgb_to_depth, rgb_bbox)
                    depth_bbox /= depth_bbox[2]  # Normalize
                    
                    # Convert bounding box coordinates to integers
                    x1, y1 = int(thermal_bbox[0, 0]), int(thermal_bbox[1, 0])  # Top-left corner for thermal
                    x2, y2 = int(thermal_bbox[0, 1]), int(thermal_bbox[1, 1])  # Bottom-right corner for thermal
                    
                    dx1, dy1 = int(depth_bbox[0, 0]), int(depth_bbox[1, 0])  # Top-left corner for depth
                    dx2, dy2 = int(depth_bbox[0, 1]), int(depth_bbox[1, 1])  # Bottom-right corner for depth

                    # Draw Bounding Boxes on images
                    cv2.rectangle(rgb_img, (x, y), (x+w, y+h), (0, 255, 0), 2)  # RGB image bbox in green
                    cv2.rectangle(thermal_img, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Thermal image bbox in blue
                    cv2.rectangle(depth_img, (dx1, dy1), (dx2, dy2), (0, 0, 255), 2)  # Depth image bbox in red
                    
                    # Show Images
                    cv2.imshow("RGB Face Detection", rgb_img)
                    cv2.imshow("Thermal Face Detection", thermal_img)
                    cv2.imshow("Depth Face Detection", depth_img)
                    
                    # Break if 'q' is pressed
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                else:
                    print(f"No faces detected in {rgb_path}")

cv2.destroyAllWindows()


Processing: RGB-01-41-30-9573
Total depth images: 9
Total thermal images: 13
Requested frame_num: 9573
Using depth_idx: 8, thermal_idx: 12
Processing: RGB-01-41-31-1557
Total depth images: 9
Total thermal images: 13
Requested frame_num: 1557
Using depth_idx: 8, thermal_idx: 12
Processing: RGB-01-41-31-2253
Total depth images: 9
Total thermal images: 13
Requested frame_num: 2253
Using depth_idx: 8, thermal_idx: 12
Processing: RGB-01-41-31-3251
Total depth images: 9
Total thermal images: 13
Requested frame_num: 3251
Using depth_idx: 8, thermal_idx: 12
Processing: RGB-01-41-31-3583
Total depth images: 9
Total thermal images: 13
Requested frame_num: 3583
Using depth_idx: 8, thermal_idx: 12
Processing: RGB-01-41-31-3912
Total depth images: 9
Total thermal images: 13
Requested frame_num: 3912
Using depth_idx: 8, thermal_idx: 12
Processing: RGB-01-41-31-4246
Total depth images: 9
Total thermal images: 13
Requested frame_num: 4246
Using depth_idx: 8, thermal_idx: 12
Processing: RGB-01-41-31-45

KeyboardInterrupt: 

In [4]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the trained model
model_best = load_model('/Users/suryanshpatel/Projects/Directed Readings/Technical/src/Models/face_model.h5') # set your machine model file path here

# Classes 7 emotional states
class_names = ['Angry', 'Disgusted', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Load the pre-trained face cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open a connection to the webcam (0 is usually the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract the face region
        face_roi = frame[y:y + h, x:x + w]

        # Resize the face image to the required input size for the model
        face_image = cv2.resize(face_roi, (48, 48))
        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
        face_image = image.img_to_array(face_image)
        face_image = np.expand_dims(face_image, axis=0)
        face_image = np.vstack([face_image])

        # Predict emotion using the loaded model
        predictions = model_best.predict(face_image)
        emotion_label = class_names[np.argmax(predictions)]

        # Display the emotion label on the frame
        cv2.putText(frame, f'Emotion: {emotion_label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, (0, 0, 255), 2)

        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Emotion Detection', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

/Users/suryanshpatel/Projects/Directed Readings/Technical/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━

KeyboardInterrupt: 

In [ ]:
model_best.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 46, 46, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 46, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 44, 44, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 44, 44, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 44, 44, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 20, 20, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 20, 20, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 18, 18, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 18, 18, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 18, 18, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 339,113 (1.29 MB)

 Trainable params: 338,407 (1.29 MB)

 Non-trainable params: 704 (2.75 KB)

 Optimizer params: 2 (12.00 B)

: 

In [27]:
import cv2

# Load the image
image_path = '/Users/suryanshpatel/Projects/Directed Readings/Technical/MintPain/Annotated_data_part1/Sub1 Daniel Simonsen/Annotated_data_Sub01_Trial01/Sub01_Trial01_Sweep01_Label0/RGB/RGB-11-51-02-0782.jpg'
image = cv2.imread(image_path)

# Check if the image was loaded successfully
if image is None:
    print("Error: Could not load image.")
else:
    # Get image dimensions
    height, width, channels = image.shape
    print(f"Image Dimensions: Width = {width}, Height = {height}")

    # Get number of channels
    print(f"Number of Channels: {channels}")

    # Get image size in bytes
    image_size = image.size
    print(f"Image Size (in bytes): {image_size}")

    # Get image data type
    image_dtype = image.dtype
    print(f"Image Data Type: {image_dtype}")


Image Dimensions: Width = 1920, Height = 1080
Number of Channels: 3
Image Size (in bytes): 6220800
Image Data Type: uint8


Image Dimensions: Width = 640, Height = 480
Number of Channels: 3
Image Size (in bytes): 921600
Image Data Type: uint8

# Generate images with labels

In [2]:
import os
import cv2
import numpy as np
from glob import glob
from pathlib import Path

# Define Paths
data_dir = "/Users/suryanshpatel/Projects/Directed Readings/Technical/MintPain/Annotated_data_part2"
subjects = sorted([subject for subject in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, subject))])

# Initialize Face Detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Pain level mapping
pain_levels = {
    "Label0": "No Pain",
    "Label1": "Mild Pain",
    "Label2": "Moderate Pain",
    "Label3": "Severe Pain",
    "Label4": "Extreme Pain"
}

# Homography Matrix Placeholder (Needs to be precomputed and loaded)
homography_rgb_to_thermal = np.eye(3)  # Replace with actual matrix
homography_rgb_to_depth = np.eye(3)  # Replace with actual matrix

for subject in subjects:
    subject_path = os.path.join(data_dir, subject)
    trials = sorted([trial for trial in os.listdir(subject_path) if os.path.isdir(os.path.join(subject_path, trial))])
    
    for trial in trials:
        trial_path = os.path.join(subject_path, trial)
        sweeps = sorted([sweep for sweep in os.listdir(trial_path) if os.path.isdir(os.path.join(trial_path, sweep))])
        
        for sweep in sweeps:
            sweep_path = os.path.join(trial_path, sweep)
            
            # Extract label from sweep folder
            label_key = sweep.split('_')[-1]  # Extracts "Label3"
            pain_label = pain_levels.get(label_key, "Unknown")
            
            # Load images
            rgb_images = sorted(glob(os.path.join(sweep_path, "RGB", "*.jpg")))
            depth_images = sorted(glob(os.path.join(sweep_path, "D", "*.png")))
            thermal_images = sorted(glob(os.path.join(sweep_path, "T", "*.png")))
            
            if len(depth_images) == 0 or len(thermal_images) == 0:
                print(f"No depth or thermal images found for sweep: {sweep_path}")
                continue
            
            for rgb_path in rgb_images:
                filename = Path(rgb_path).stem
                frame_num = int(filename.split('-')[-1])  # Adjust parsing if needed
                
                depth_idx = min(frame_num, len(depth_images) - 1)
                thermal_idx = min(frame_num, len(thermal_images) - 1)
                
                depth_path = depth_images[depth_idx]
                thermal_path = thermal_images[thermal_idx]
                
                if not os.path.exists(depth_path) or not os.path.exists(thermal_path):
                    print(f"Missing depth or thermal image for {rgb_path}")
                    continue
                
                rgb_img = cv2.imread(rgb_path)
                depth_img = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)
                thermal_img = cv2.imread(thermal_path, cv2.IMREAD_GRAYSCALE)
                
                gray_rgb = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray_rgb, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
                
                if len(faces) > 0:
                    x, y, w, h = faces[0]  # Assuming first face is the correct one
                    
                    # Project bounding box using homography
                    rgb_bbox = np.array([[x, y, 1], [x+w, y+h, 1]]).T
                    
                    thermal_bbox = np.dot(homography_rgb_to_thermal, rgb_bbox)
                    thermal_bbox /= thermal_bbox[2]
                    
                    depth_bbox = np.dot(homography_rgb_to_depth, rgb_bbox)
                    depth_bbox /= depth_bbox[2]
                    
                    # Convert bounding box coordinates to integers
                    x1, y1 = int(thermal_bbox[0, 0]), int(thermal_bbox[1, 0])
                    x2, y2 = int(thermal_bbox[0, 1]), int(thermal_bbox[1, 1])
                    
                    dx1, dy1 = int(depth_bbox[0, 0]), int(depth_bbox[1, 0])
                    dx2, dy2 = int(depth_bbox[0, 1]), int(depth_bbox[1, 1])
                    
                    # Draw bounding boxes
                    cv2.rectangle(rgb_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.rectangle(thermal_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.rectangle(depth_img, (dx1, dy1), (dx2, dy2), (0, 0, 255), 2)
                    
                    # Overlay pain label on RGB image
                    cv2.putText(rgb_img, f"Pain Level: {pain_label}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                                1, (0, 0, 255), 2, cv2.LINE_AA)
                    
                    # Display images
                    cv2.imshow("RGB Face Detection", rgb_img)
                    cv2.imshow("Thermal Face Detection", thermal_img)
                    cv2.imshow("Depth Face Detection", depth_img)
                    
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                else:
                    print(f"No faces detected in {rgb_path}")

cv2.destroyAllWindows()


KeyboardInterrupt: 